<a href="https://colab.research.google.com/github/hugomaharavo/ai4eng-20242-Pruebas-Saber-Pro-Colombia/blob/main/04%20-%20Modelo%20con%20preprocesado%20de%20todas%20las%20columnas%20de%20train%20y%20test%20en%20la%20misma%20cedula%20y%20Modelo%20RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ACCEDER A LOS ARCHOIVOS EN KAGGLE
# 1 - Instale la biblioteca opendatasets para descargar conjuntos de datos directamente de Kaggle
!pip install opendatasets

# 2 - Importar los módulos necesarios
import opendatasets as kaggle_data
import os

# 3 - Enlace al concurso de Kaggle
competition_url = "https://www.kaggle.com/competitions/udea-ai4eng-20242"

# 4 - Descargar los datos del concurso
kaggle_data.download(competition_url)

# 5 - Cambiar de directorio para acceder a la carpeta del conjunto de datos descargado
project_dir = "udea-ai4eng-20242"
os.chdir(project_dir)

# 6 - Comprobación de los archivos descargados
files = os.listdir()
print("Fichiers disponibles dans le dossier : ", files)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: hugomaharavo
Your Kaggle Key: ··········


100%|██████████| 20.1M/20.1M [00:00<00:00, 90.6MB/s]



Extracting archive ./udea-ai4eng-20242/udea-ai4eng-20242.zip to ./udea-ai4eng-20242
Fichiers disponibles dans le dossier :  ['train.csv', 'test.csv', 'submission_example.csv']


In [ ]:
# IMPORTER LES LIBRAIRIES
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# 2 - CARGAR LOS DATOS INICIALES
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# 3 - APLICAR CORRESPONDENCIAS Y CONVERSIONES
# Convertir valores ordinales en numéricos
tuition_average_mapping = {
    "No pagó matrícula": 0.0, "Menos de 500 mil": 0.25, "Entre 500 mil y menos de 1 millón": 0.75,
    "Entre 1 millón y menos de 2.5 millones": 1.75, "Entre 2.5 millones y menos de 4 millones": 3.25,
    "Entre 4 millones y menos de 5.5 millones": 4.75, "Entre 5.5 millones y menos de 7 millones": 6.25, "Más de 7 millones": 7.5
}
train_df['ESTU_VALORMATRICULAUNIVERSIDAD'] = train_df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(tuition_average_mapping)
test_df['ESTU_VALORMATRICULAUNIVERSIDAD'] = test_df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(tuition_average_mapping)

# Codificación de FAMI_ESTRATOVIVIENDA
housing_strata_mapping = {"Estrato 1": 1, "Estrato 2": 2, "Estrato 3": 3, "Estrato 4": 4, "Estrato 5": 5, "Estrato 6": 6, "Sin Estrato": None}
train_df['FAMI_ESTRATOVIVIENDA'] = train_df['FAMI_ESTRATOVIVIENDA'].map(housing_strata_mapping)
test_df['FAMI_ESTRATOVIVIENDA'] = test_df['FAMI_ESTRATOVIVIENDA'].map(housing_strata_mapping)

# Codificación de FAMI_TIENEINTERNET y ESTU_PAGOMATRICULAPROPIO
binary_mappings = {"Si": 1, "No": 0}
train_df['FAMI_TIENEINTERNET'] = train_df['FAMI_TIENEINTERNET'].map(binary_mappings)
test_df['FAMI_TIENEINTERNET'] = test_df['FAMI_TIENEINTERNET'].map(binary_mappings)
train_df['ESTU_PAGOMATRICULAPROPIO'] = train_df['ESTU_PAGOMATRICULAPROPIO'].map(binary_mappings)
test_df['ESTU_PAGOMATRICULAPROPIO'] = test_df['ESTU_PAGOMATRICULAPROPIO'].map(binary_mappings)

# Codificación de la variable de destino
target_mapping = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
train_df['RENDIMIENTO_GLOBAL'] = train_df['RENDIMIENTO_GLOBAL'].map(target_mapping)

# 4 - SEPARAR LAS CARACTERÍSTICAS Y EL OBJETIVO
X_train = train_df.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
y_train = train_df['RENDIMIENTO_GLOBAL']
X_test = test_df.drop(columns=['ID'])

# 5 - PROCESO PARA PRETRATAMIENTO
numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

# Proceso previo con OneHotEncoder para variables categóricas e imputación de valores perdidos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ]
)

# 6 - ENTRENAR EL MODELO
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))])

pipeline.fit(X_train, y_train)

# 7 - HACER PREDICCIONES SOBRE EL CONJUNTO DE DATOS test.csv
predictions = pipeline.predict(X_test)

# 8 - CONVERTIR LAS PREDICCIONES EN PALABRAS ORIGINALES
target_inverse_mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
predictions_labels = [target_inverse_mapping[pred] for pred in predictions]

# 9 - CREAR EL ARCHIVO DE SUMISIÓN
submission_df = pd.DataFrame({
    "ID": test_df['ID'],
    "RENDIMIENTO_GLOBAL": predictions_labels
})
submission_df.to_csv("submission.csv", index=False)
